In [1]:
from matplotlib.colors import ListedColormap
from PIL import Image
import logging
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import numpy as np
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from tiatoolbox.tools.patchextraction import SlidingWindowPatchExtractor
from tiatoolbox.wsicore.wsireader import WSIReader

|2025-07-18|21:36:45.689| [WARNING] /cluster/home/srivash/miniforge3/envs/tiatoolbox/lib/python3.10/site-packages/albumentations/__init__.py:24: UserWarning: A new version of Albumentations is available: 2.0.8 (you have 1.4.21). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()



In [5]:
from huggingface_hub import login


In [6]:
login()

In [7]:
import timm
import torch
from timm.data import resolve_data_config
from timm.data.transforms_factory import create_transform
from timm.layers import SwiGLUPacked
from PIL import Image

# need to specify MLP layer and activation function for proper init
model = timm.create_model("hf-hub:paige-ai/Virchow2", pretrained=True, mlp_layer=SwiGLUPacked, act_layer=torch.nn.SiLU)
model = model.eval()

KeyboardInterrupt: 

In [2]:
roi_img_dir = '/lab/deasylab3/Himangi/tnbc_data/roi_imagesroi_TCGA-A1-A0SP-01Z-00-DX1.20D689C6-EFA5-4694-BE76-24475A89ACC0_3000_24862_9000_30896.png'

In [3]:
reader = WSIReader.open(roi_img_dir)
print(reader)  # noqa: T201

In [4]:
from tiatoolbox.tools.patchextraction import SlidingWindowPatchExtractor
from tiatoolbox.utils.misc import imread, imwrite
import matplotlib.pyplot as plt
import os
import csv

# 1. Load image from path
input_img = imread(roi_img_dir)  # roi_img_dir must be a valid image path

# 2. Patch and stride settings
patch_size = (224, 224)
stride = (224, 224)

# 3. Image dimensions
height, width = input_img.shape[:2]

# 4. Create extractor
extractor = SlidingWindowPatchExtractor(
    input_img=input_img,
    patch_size=patch_size,
    stride=stride,
)

# 5. Create output folder and CSV
os.makedirs("tiles", exist_ok=True)

csv_path = "tiles/patch_metadata.csv"
with open(csv_path, "w", newline="") as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["Index", "X", "Y", "Filename"])

    i = 0
    for y in range(0, height - patch_size[1] + 1, stride[1]):
        for x in range(0, width - patch_size[0] + 1, stride[0]):
            patch = extractor[i]
            i += 1

            filename = f"patch_{i:04d}_x{x}_y{y}.png"
            imwrite(os.path.join("tiles", filename), patch)
            writer.writerow([i, x, y, filename])

print("✅ Done — CSV should be at:", os.path.abspath(csv_path))

|2025-07-14|11:17:54.118| [WARNING] Raw data is None.
|2025-07-14|11:17:54.119| [WARNING] Unknown scale (no objective_power or mpp)


✅ Done — CSV should be at: /lab/deasylab3/Himangi/TNBC_multimodal/tiles/patch_metadata.csv


In [5]:
# Clear logger to use tiatoolbox.logger
import logging
import warnings

import cv2
import joblib
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np

from tiatoolbox import logger
from tiatoolbox.models.engine.nucleus_instance_segmentor import NucleusInstanceSegmentor
from tiatoolbox.utils.misc import download_data, imread

# We need this function to visualize the nuclear predictions
from tiatoolbox.utils.visualization import (
    overlay_prediction_contours,
)
from tiatoolbox.wsicore.wsireader import WSIReader

if logging.getLogger().hasHandlers():
    logging.getLogger().handlers.clear()
warnings.filterwarnings("ignore")
mpl.rcParams["figure.dpi"] = 300  # for high resolution figure in notebook
mpl.rcParams["figure.facecolor"] = "white"  # To make sure text is visible in dark mode
plt.rcParams.update({"font.size": 5})
import glob

In [6]:



# cell segmentation 
inst_segmentor = NucleusInstanceSegmentor(
    pretrained_model="hovernet_fast-pannuke",
    num_loader_workers=2,
    num_postproc_workers=2,
    batch_size=4,
)


In [7]:
png_paths = sorted(glob.glob(os.path.join("tiles", "*.png")))



In [ ]:


if os.path.exists(TEST_FOLDER):
    shutil.rmtree(TEST_FOLDER)
os.makedirs(TEST_FOLDER)

tile_output = inst_segmentor.predict(
    png_paths,
    save_dir="sample_tile_results3/",
    mode="tile",
    device="cuda",
    crash_on_exception=False,
)

In [32]:
tile_output = inst_segmentor.predict(
    png_paths,
    save_dir="sample_tile_results3/",
    mode="tile",
    device="cuda",
    crash_on_exception=False,
)

ValueError: `save_dir` already exists! /lab/deasylab3/Himangi/TNBC_multimodal/sample_tile_results3

In [41]:
tile_preds = joblib.load(f"{tile_output[550]}.dat")


FileNotFoundError: [Errno 2] No such file or directory: "['tiles/patch_0551_x896_y4704.png', '/lab/deasylab3/Himangi/TNBC_multimodal/sample_tile_results3/550'].dat"

In [48]:
tile_preds = joblib.load(f"{tile_output[555][1]}.dat")
logger.info(f"Number of detected nuclei: {len(tile_preds)}")

# Extracting the nucleus IDs and select the first one
nuc_id_list = list(tile_preds.keys())


INFO:root:Number of detected nuclei: 32


In [104]:
import os

dat_folder = "sample_tile_results3"

# List all .dat files not starting with 'file'
dat_files = [
    f for f in os.listdir(dat_folder)
    if f.endswith(".dat") and not f.startswith("file")
]

print(f"Found {len(dat_files)} .dat files (excluding those starting with 'file'):")
print(dat_files)


Found 676 .dat files (excluding those starting with 'file'):
['0.dat', '1.dat', '2.dat', '3.dat', '4.dat', '5.dat', '6.dat', '7.dat', '8.dat', '9.dat', '10.dat', '11.dat', '12.dat', '13.dat', '14.dat', '15.dat', '16.dat', '17.dat', '18.dat', '19.dat', '20.dat', '21.dat', '22.dat', '23.dat', '24.dat', '25.dat', '26.dat', '27.dat', '28.dat', '29.dat', '30.dat', '31.dat', '32.dat', '33.dat', '34.dat', '35.dat', '36.dat', '37.dat', '38.dat', '39.dat', '40.dat', '41.dat', '42.dat', '43.dat', '44.dat', '45.dat', '46.dat', '47.dat', '48.dat', '49.dat', '50.dat', '51.dat', '52.dat', '53.dat', '54.dat', '55.dat', '56.dat', '57.dat', '58.dat', '59.dat', '60.dat', '61.dat', '62.dat', '63.dat', '64.dat', '65.dat', '66.dat', '67.dat', '68.dat', '69.dat', '70.dat', '71.dat', '72.dat', '73.dat', '74.dat', '75.dat', '76.dat', '77.dat', '78.dat', '79.dat', '80.dat', '81.dat', '82.dat', '83.dat', '84.dat', '85.dat', '86.dat', '87.dat', '88.dat', '89.dat', '90.dat', '91.dat', '92.dat', '93.dat', '94.dat'

In [101]:
import joblib
import pandas as pd
import glob


# Mapping of type labels
type_mapping = {
    0: "Background",
    1: "Tumor",
    2: "Immune",
    3: "Inflammatory",
    4: "Connective",
    5: "Dead"
}
Master_nuclei_data = []

for data in dat_files:
    file_number = os.path.splitext(data)[0]  # e.g., '555' from '555.dat'
    try:
        tile_preds = joblib.load(data)
    except Exception as e:
        print(f"Failed to load {data}: {e}")
        continue

    # Load a single .dat file
    tile_dat_file = "sample_tile_results3/555.dat"
    tile_preds = joblib.load(tile_dat_file)

    Master_nuclei_data = []

    
    nuc_id_list = list(tile_preds.keys())
    
    for i in range(len(nuc_id_list)):
        selected_nuc_id = nuc_id_list[i]
        sample_nuc = tile_preds[selected_nuc_id]
    
        nuc_id = selected_nuc_id
        cell_type_id = sample_nuc["type"]
        cell_type_name = type_mapping.get(cell_type_id, "Unknown")
    
        bounding_box = [
            sample_nuc["box"][0],
            sample_nuc["box"][1],
            sample_nuc["box"][2],
            sample_nuc["box"][3]
        ]
    
        centroid = [
            sample_nuc["centroid"][0],
            sample_nuc["centroid"][1]
        ]
    
        probability = sample_nuc["prob"]
    
        nuclei_data.append({
            "nuc_id": nuc_id,
            "type": cell_type_id,
            "type_name": cell_type_name,
            "bounding_box": bounding_box,
            "centroid": centroid,
            "prob": probability
        })

# Convert to DataFrame
df = pd.DataFrame(nuclei_data)

# Optional: save to CSV
# df.to_csv("tile_555_nuclei_data.csv", index=False)

# Show preview
print(df.head())


                             nuc_id  type type_name        bounding_box  \
0  5772d1d5819e4685bd306c1b7cb12c67     1     Tumor  [134, 39, 151, 56]   
1  56a9bb390ca74e2ca64eb9dfb4dbcec0     1     Tumor    [36, 42, 66, 62]   
2  48f6b51d11894dfe806da774b31ff851     1     Tumor   [99, 55, 127, 71]   
3  6c3f6c9722ca41f5860f8c123f50da2e     1     Tumor  [135, 62, 150, 80]   
4  7abb3f9c76814a079cbe7328fb474d4e     1     Tumor  [154, 64, 174, 82]   

                                  centroid      prob  
0  [142.0990990990991, 47.387387387387385]  0.972973  
1   [49.3448275862069, 50.716180371352785]  0.870027  
2  [112.22402597402598, 62.40909090909091]  0.957792  
3  [142.08415841584159, 70.03960396039604]  0.910891  
4    [164.0326530612245, 72.2204081632653]  0.979592  


In [106]:
import os
import joblib
import pandas as pd

# Folder path
dat_folder = "sample_tile_results3"

# Get all .dat files excluding ones starting with "file"
dat_files = [
    os.path.join(dat_folder, f)
    for f in os.listdir(dat_folder)
    if f.endswith(".dat") and not f.startswith("file")
]

# Mapping of type labels
type_mapping = {
    0: "Background",
    1: "Tumor",
    2: "Immune",
    3: "Inflammatory",
    4: "Connective",
    5: "Dead"
}

# Master list to collect all nuclei data
Master_nuclei_data = []

# Loop over each valid .dat file
for data in dat_files:
    file_number = os.path.splitext(os.path.basename(data))[0]  # e.g., '555'

    try:
        tile_preds = joblib.load(data)
    except Exception as e:
        print(f"Failed to load {data}: {e}")
        continue

    nuc_id_list = list(tile_preds.keys())

    for nuc_id in nuc_id_list:
        sample_nuc = tile_preds[nuc_id]

        cell_type_id = sample_nuc.get("type")
        cell_type_name = type_mapping.get(cell_type_id, "Unknown")

        bounding_box = sample_nuc.get("box", [None, None, None, None])
        centroid = sample_nuc.get("centroid", [None, None])
        sample_nuc["contour"]
        probability = sample_nuc.get("prob")

        Master_nuclei_data.append({
            "file_number": file_number,
            "nuc_id": nuc_id,
            "type": cell_type_id,
            "type_name": cell_type_name,
            "bounding_box": bounding_box,
            "centroid": centroid,
            
            "prob": probability
        })

# Convert to DataFrame
df = pd.DataFrame(Master_nuclei_data)

# Optional: save
df.to_csv("all_nuclei_data.csv", index=False)

print(f"Total nuclei processed: {len(df)}")
print(df.head())


Total nuclei processed: 27743
  file_number                            nuc_id  type     type_name  \
0           0  ebe6bc1ea6b544aca2786a5e7c73bb4e     3  Inflammatory   
1           0  9fbd4c54b7d74e98b633cec629eb8d4a     3  Inflammatory   
2           1  9c4cfd1a3b7642c9a0288de6382ae789     3  Inflammatory   
3           1  6f90c615363c4632a3d851f3cbe83ee4     3  Inflammatory   
4           1  13ff7f24d6a7417aaf3f7bfbd39ca551     3  Inflammatory   

           bounding_box                                  centroid      prob  
0    [117, 39, 126, 56]   [121.00869565217391, 47.19130434782609]  1.000000  
1    [174, 62, 184, 76]   [178.43119266055047, 68.98165137614679]  0.917431  
2      [17, 19, 28, 52]   [21.835748792270532, 35.56038647342995]  0.990338  
3    [133, 37, 144, 58]    [137.63522012578616, 46.9496855345912]  0.987421  
4  [178, 149, 189, 171]  [182.94160583941607, 158.43065693430657]  1.000000  


In [108]:
import os
import joblib
import pandas as pd

# Folder path
dat_folder = "sample_tile_results3"

# Get all .dat files excluding those starting with "file"
dat_files = [
    os.path.join(dat_folder, f)
    for f in os.listdir(dat_folder)
    if f.endswith(".dat") and not f.startswith("file")
]

# Mapping of type labels
type_mapping = {
    0: "Background",
    1: "Tumor",
    2: "Immune",
    3: "Inflammatory",
    4: "Connective",
    5: "Dead"
}

# Master list to collect all nuclei data
Master_nuclei_data = []

# Loop over each valid .dat file
for data in dat_files:
    file_number = os.path.splitext(os.path.basename(data))[0]  # e.g., '555'

    try:
        tile_preds = joblib.load(data)
    except Exception as e:
        print(f"Failed to load {data}: {e}")
        continue

    for nuc_id, sample_nuc in tile_preds.items():
        cell_type_id = sample_nuc.get("type")
        cell_type_name = type_mapping.get(cell_type_id, "Unknown")

        bounding_box = sample_nuc.get("box", [None, None, None, None])
        centroid = sample_nuc.get("centroid", [None, None])
        probability = sample_nuc.get("prob")

        # Convert contour to plain list of [x, y]
        raw_contour = sample_nuc.get("contour", [])
        contour = [list(point) for point in raw_contour]  # Ensure it's JSON/CSV serializable

        Master_nuclei_data.append({
            "file_number": file_number,
            "nuc_id": nuc_id,
            "type": cell_type_id,
            "type_name": cell_type_name,
            "bounding_box": bounding_box,
            "centroid": centroid,
            "prob": probability,
            "contour": contour
        })

# Convert to DataFrame
df = pd.DataFrame(Master_nuclei_data)

# Optional: Save to CSV (convert contour to string format for CSV)
df.to_csv("all_nuclei_data_with_contour.csv", index=False)

# Display result
print(f"Total nuclei processed: {len(df)}")
print(df.head())


Total nuclei processed: 27743
  file_number                            nuc_id  type     type_name  \
0           0  ebe6bc1ea6b544aca2786a5e7c73bb4e     3  Inflammatory   
1           0  9fbd4c54b7d74e98b633cec629eb8d4a     3  Inflammatory   
2           1  9c4cfd1a3b7642c9a0288de6382ae789     3  Inflammatory   
3           1  6f90c615363c4632a3d851f3cbe83ee4     3  Inflammatory   
4           1  13ff7f24d6a7417aaf3f7bfbd39ca551     3  Inflammatory   

           bounding_box                                  centroid      prob  \
0    [117, 39, 126, 56]   [121.00869565217391, 47.19130434782609]  1.000000   
1    [174, 62, 184, 76]   [178.43119266055047, 68.98165137614679]  0.917431   
2      [17, 19, 28, 52]   [21.835748792270532, 35.56038647342995]  0.990338   
3    [133, 37, 144, 58]    [137.63522012578616, 46.9496855345912]  0.987421   
4  [178, 149, 189, 171]  [182.94160583941607, 158.43065693430657]  1.000000   

                                             contour  
0  [[119, 39]

In [112]:
import os
import cv2
import numpy as np
import joblib
import re

# --- CONFIGURATION ---
dat_folder = "sample_tile_results3"       # Folder with .dat files
tile_image_folder = "tiles"         # Folder with original tile images (224x224)
overlay_folder = "/lab/deasylab3/Himangi/tnbc_data/tile_overlays"          # Output folder for contour overlays
output_path = "/lab/deasylab3/Himangi/tnbc_data/stitched_roi_with_tumor_contours.png"  # Final stitched ROI
TILE_SIZE = 224
TUMOR_TYPE_ID = 1

os.makedirs(overlay_folder, exist_ok=True)

# --- STEP 1: Get matching .png and .dat files sorted ---
dat_files = sorted(
    [f for f in os.listdir(dat_folder) if f.endswith(".dat") and not f.startswith("file")],
    key=lambda x: int(os.path.splitext(x)[0])
)

png_files = sorted(
    [f for f in os.listdir(tile_image_folder) if f.endswith(".png") and "x" in f and "y" in f],
    key=lambda x: int(re.search(r"patch_(\d+)", x).group(1))
)

if len(dat_files) != len(png_files):
    raise ValueError(f"Mismatch: {len(dat_files)} .dat files vs {len(png_files)} .png tiles")

tile_positions = {}
max_x = max_y = 0

# --- STEP 2: Process each tile and overlay contours ---
print("🔄 Generating tumor overlays from matched files...")

for idx, (dat_file, png_file) in enumerate(zip(dat_files, png_files)):
    dat_path = os.path.join(dat_folder, dat_file)
    png_path = os.path.join(tile_image_folder, png_file)
    tile_id = os.path.splitext(png_file)[0]
    out_path = os.path.join(overlay_folder, f"{tile_id}.png")

    # Extract x/y from PNG filename
    match = re.search(r"x(\d+)_y(\d+)", png_file)
    if not match:
        print(f"⚠️ Skipping malformed filename: {png_file}")
        continue

    x_offset = int(match.group(1))
    y_offset = int(match.group(2))

    tile_positions[tile_id] = (x_offset, y_offset)
    max_x = max(max_x, x_offset)
    max_y = max(max_y, y_offset)

    try:
        preds = joblib.load(dat_path)
        img = cv2.imread(png_path)
    except Exception as e:
        print(f"❌ Error loading {dat_file} or {png_file}: {e}")
        continue

    overlay = img.copy()

    for nuc_id, nuc in preds.items():
        if not isinstance(nuc, dict) or nuc.get("type") != TUMOR_TYPE_ID:
            continue

        contour = np.array(nuc.get("contour", []), dtype=np.int32)
        if contour.size == 0:
            continue

        contour = contour.reshape((-1, 1, 2))
        cv2.drawContours(overlay, [contour], -1, (255, 0, 0), 1)

    cv2.imwrite(out_path, overlay)

print("✅ Tumor overlays saved.\n")

# --- STEP 3: Stitch overlays using (x_offset, y_offset) ---
stitched_width = max_x + TILE_SIZE
stitched_height = max_y + TILE_SIZE
stitched = np.zeros((stitched_height, stitched_width, 3), dtype=np.uint8)

print("🧩 Stitching tiles into full ROI...")

for tile_id, (x_offset, y_offset) in tile_positions.items():
    tile_path = os.path.join(overlay_folder, f"{tile_id}.png")
    if not os.path.exists(tile_path):
        continue

    tile_img = cv2.imread(tile_path)
    stitched[y_offset:y_offset+TILE_SIZE, x_offset:x_offset+TILE_SIZE] = tile_img

cv2.imwrite(output_path, stitched)
print(f"✅ Full ROI with tumor contours saved to:\n➡️ {output_path}")


🔄 Generating tumor overlays from matched files...
✅ Tumor overlays saved.

🧩 Stitching tiles into full ROI...
✅ Full ROI with tumor contours saved to:
➡️ /lab/deasylab3/Himangi/tnbc_data/stitched_roi_with_tumor_contours.png


In [91]:
tile_dat_file = "sample_tile_results3/555.dat"
tile_preds = joblib.load(tile_dat_file)
nuc_id_list = list(tile_preds.keys())

In [85]:
import joblib
import logging

logger = logging.getLogger()

# Load .dat file from HoverNet output
tile_dat_file = "sample_tile_results3/555.dat"
tile_preds = joblib.load(tile_dat_file)

logger.info(f"Number of detected nuclei: {len(tile_preds)}")

# Pick the first nucleus
nuc_id_list = list(tile_preds.keys())
selected_nuc_id = nuc_id_list[12]

sample_nuc = tile_preds[selected_nuc_id]
sample_nuc_keys = list(sample_nuc.keys())

logger.info(
    "Keys in the output dictionary: [%s]",
    ", ".join(sample_nuc_keys)
)

logger.info(
    "Bounding box: (%d, %d, %d, %d)",
    sample_nuc["box"][0],
    sample_nuc["box"][1],
    sample_nuc["box"][2],
    sample_nuc["box"][3],
)

logger.info(
    "Centroid: (%d, %d)",
    sample_nuc["centroid"][0],
    sample_nuc["centroid"][1],
)

logger.info(
    "Type label: %s",
    sample_nuc["type"]
)


logger.info(
    "Type contour: %s",
    sample_nuc["contour"]
)


logger.info(
    "prob: %s",
    sample_nuc["prob"]
)

INFO:root:Number of detected nuclei: 32
INFO:root:Keys in the output dictionary: [box, centroid, contour, prob, type]
INFO:root:Bounding box: (70, 95, 87, 112)
INFO:root:Centroid: (77, 102)
INFO:root:Type label: 1
INFO:root:Type contour: [[ 78  95]
 [ 77  96]
 [ 76  96]
 [ 75  97]
 [ 74  97]
 [ 73  98]
 [ 73  99]
 [ 70 102]
 [ 70 108]
 [ 73 111]
 [ 74 111]
 [ 75 110]
 [ 78 110]
 [ 79 109]
 [ 80 109]
 [ 83 106]
 [ 83 105]
 [ 84 104]
 [ 84 103]
 [ 85 102]
 [ 85 101]
 [ 86 100]
 [ 86  97]
 [ 85  97]
 [ 83  95]]
INFO:root:prob: 0.9607843090157632


In [86]:
import os
import joblib
import pandas as pd

# Folder containing .dat files
dat_folder = "sample_tile_results3"

# Initialize list to hold all rows
nuclei_data = []

# Iterate through all .dat files
for filename in os.listdir(dat_folder):
    if filename.endswith(".dat"):
        filepath = os.path.join(dat_folder, filename)
        tile_id = os.path.splitext(filename)[0]  # e.g., '555' from '555.dat'

        try:
            tile_preds = joblib.load(filepath)
        except Exception as e:
            print(f"Failed to load {filename}: {e}")
            continue

        for nuc_id, nuc_info in tile_preds.items():
            row = {
                "file_number": tile_id,
                "nuc_id": nuc_id,
                "type": nuc_info.get("type"),
                "bounding_box": tuple(nuc_info.get("box", (None, None, None, None))),
                "contour": nuc_info.get("contour"),
                "prob": nuc_info.get("prob")
            }
            nuclei_data.append(row)

# Convert to DataFrame
df = pd.DataFrame(nuclei_data)

# Optional: Save to CSV
# df.to_csv("nuclei_summary.csv", index=False)

print(f"Loaded data for {len(df)} nuclei across {df['file_number'].nunique()} tiles.")


AttributeError: 'list' object has no attribute 'items'

In [60]:
tumor_class = 1  # tumor in PanNuke

tumor_nuclei = {
    nid: info
    for nid, info in tile_preds.items()
    if info["type"] == tumor_class
}

logger.info(f"Tumor nuclei count: {len(tumor_nuclei)}")


INFO:root:Tumor nuclei count: 26


In [66]:
tumor_patches = []
for nid, info in tumor_nuclei.items():
    x, y = info["centroid"]
    x1 = max(0, x - 224 // 2)
    y1 = max(0, y - 224 // 2)
    x2 = x1 + 224
    y2 = y1 + 224


In [73]:
142.0990991-224 // 2

30.09909909999999

In [76]:

for nid, info in tumor_nuclei.items():
    x, y = info["centroid"]
    x1 = max(0, x - 224 // 2)
    y1 = max(0, y - 224 // 2)
    print(x1, y1)

30.099099099099107 0
0 0
0.22402597402597735 0
30.084158415841586 0
52.032653061224494 0
72.15517241379311 0
0 0
0 0
0 0
13.087855297157617 0
52.74038461538461 0
0 0
9.64577656675749 8.460490463215251
0 3.138686131386862
35.58019801980197 14.986138613861385
0 19.73975903614459
0 21.68965517241378
58.542124542124554 24.849816849816847
0 39.206303724928375
0 55.926315789473676
19.30857142857144 60.54857142857142
0 64.0811965811966
10.302238805970148 76.04850746268656
0 76.74799999999999
0 93.7439024390244
0 98.0084745762712
